# Import Event Log

In [1]:
import pandas as pd
import numpy as np
import pm4py
from pm4py.objects.conversion.log import converter as log_converter

if __name__ == "__main__":
    # Read the XES file
    dataframe_log = pm4py.read_xes('../../data/logs/InternationalDeclarations.xes')

    # If 'log' is already a DataFrame, add the @@case_index column directly
    case_indices = {case_id: idx for idx, case_id in enumerate(dataframe_log['case:concept:name'].unique())}
    dataframe_log['@@case_index'] = dataframe_log['case:concept:name'].map(case_indices)
    
     # Convert the dataframe to event log
    log = log_converter.apply(dataframe_log)
    
dataframe_log

/opt/anaconda3/lib/python3.9/site-packages/pm4py/util/dt_parsing/parser.py:76: UserWarning: ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11
  warnings.warn(


parsing log, completed traces ::   0%|          | 0/6449 [00:00<?, ?it/s]

,id,org:resource,concept:name,time:timestamp,org:role,case:Permit travel permit number,case:DeclarationNumber,case:Amount,case:RequestedAmount,case:Permit TaskNumber,...,case:Permit OrganizationalEntity,case:travel permit number,case:Permit RequestedBudget,case:id,case:Permit ID,case:Permit id,case:BudgetNumber,case:Permit ActivityNumber,case:AdjustedAmount,@@case_index
0,rv_travel permit 76455_6,STAFF MEMBER,Start trip,2016-10-04 22:00:00+00:00,EMPLOYEE,travel permit number 76456,declaration number 76458,39.664561,39.664561,UNKNOWN,...,organizational unit 65458,travel permit number 76456,41.613445,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,0
1,rv_travel permit 76455_7,STAFF MEMBER,End trip,2016-10-04 22:00:00+00:00,EMPLOYEE,travel permit number 76456,declaration number 76458,39.664561,39.664561,UNKNOWN,...,organizational unit 65458,travel permit number 76456,41.613445,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,0
2,st_step 76459_0,STAFF MEMBER,Permit SUBMITTED by EMPLOYEE,2017-04-06 11:32:10+00:00,EMPLOYEE,travel permit number 76456,declaration number 76458,39.664561,39.664561,UNKNOWN,...,organizational unit 65458,travel permit number 76456,41.613445,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,0
3,st_step 76460_0,STAFF MEMBER,Permit FINAL_APPROVED by SUPERVISOR,2017-04-06 11:32:28+00:00,SUPERVISOR,travel permit number 76456,declaration number 76458,39.664561,39.664561,UNKNOWN,...,organizational unit 65458,travel permit number 76456,41.613445,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,0
4,st_step 76461_0,STAFF MEMBER,Declaration SUBMITTED by EMPLOYEE,2017-04-07 11:38:14+00:00,EMPLOYEE,travel permit number 76456,declaration number 76458,39.664561,39.664561,UNKNOWN,...,organizational unit 65458,travel permit number 76456,41.613445,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72146,st_step 13239_0,STAFF MEMBER,Declaration SUBMITTED by EMPLOYEE,2018-12-18 14:06:50+00:00,EMPLOYEE,travel permit number 13227,UNKNOWN,0.000000,0.000000,task 427,...,organizational unit 65455,travel permit number 13227,1727.559756,declaration 13232,travel permit 13226,travel permit 13226,budget 147449,UNKNOWN,0.000000,6448
72147,st_step 13241_0,STAFF MEMBER,Declaration REJECTED by ADMINISTRATION,2018-12-18 14:06:57+00:00,ADMINISTRATION,travel permit number 13227,UNKNOWN,0.000000,0.000000,task 427,...,organizational unit 65455,travel permit number 13227,1727.559756,declaration 13232,travel permit 13226,travel permit 13226,budget 147449,UNKNOWN,0.000000,6448
72148,st_step 13240_0,STAFF MEMBER,Declaration REJECTED by EMPLOYEE,2018-12-19 13:05:36+00:00,EMPLOYEE,travel permit number 13227,UNKNOWN,0.000000,0.000000,task 427,...,organizational unit 65455,travel permit number 13227,1727.559756,declaration 13232,travel permit 13226,travel permit 13226,budget 147449,UNKNOWN,0.000000,6448
72149,rv_travel permit 13226_6,STAFF MEMBER,Start trip,2019-02-18 23:00:00+00:00,EMPLOYEE,travel permit number 13227,UNKNOWN,0.000000,0.000000,task 427,...,organizational unit 65455,travel permit number 13227,1727.559756,declaration 13232,travel permit 13226,travel permit 13226,budget 147449,UNKNOWN,0.000000,6448


# Drop unnessary columns

In [2]:
dataframe_log = dataframe_log.drop(columns=['case:Permit ActivityNumber'])
dataframe_log = dataframe_log.drop(columns=['case:BudgetNumber'])
dataframe_log = dataframe_log.drop(columns=['case:Permit ID'])
dataframe_log = dataframe_log.drop(columns=['case:id'])
dataframe_log = dataframe_log.drop(columns=['case:travel permit number'])
dataframe_log = dataframe_log.drop(columns=['case:concept:name'])
dataframe_log = dataframe_log.drop(columns=['case:Permit ProjectNumber'])
dataframe_log = dataframe_log.drop(columns=['case:Permit TaskNumber'])
dataframe_log = dataframe_log.drop(columns=['case:DeclarationNumber'])
dataframe_log = dataframe_log.drop(columns=['case:Permit travel permit number'])
dataframe_log = dataframe_log.drop(columns=['case:Amount'])
dataframe_log = dataframe_log.drop(columns=['case:OriginalAmount'])
dataframe_log = dataframe_log.drop(columns=['case:AdjustedAmount'])
dataframe_log = dataframe_log.drop(columns=['case:Permit id'])
dataframe_log = dataframe_log.drop(columns=['case:Permit RequestedBudget'])
dataframe_log = dataframe_log.drop(columns=['case:Permit OrganizationalEntity'])
dataframe_log = dataframe_log.drop(columns=['case:Permit BudgetNumber'])
dataframe_log = dataframe_log.drop(columns=['case:RequestedAmount'])

# Preprocess

In [3]:
from sklearn.preprocessing import StandardScaler

# Convert to datetime format
dataframe_log['time:timestamp'] = pd.to_datetime(dataframe_log['time:timestamp'])

# Calculate elapsed time since the start of each case
dataframe_log['start_time'] = dataframe_log.groupby('@@case_index')['time:timestamp'].transform('min')
dataframe_log['elapsed_time'] = (dataframe_log['time:timestamp'] - dataframe_log['start_time']).dt.total_seconds()

# Normalize the elapsed time in minutes
scaler = StandardScaler()
dataframe_log['standardized_elapsed_time'] = scaler.fit_transform(dataframe_log[['elapsed_time']])

dataframe_log = dataframe_log.drop(columns=['start_time'])
dataframe_log = dataframe_log.drop(columns=['elapsed_time'])
dataframe_log = dataframe_log.drop(columns=['time:timestamp'])

In [4]:
codes, uniques = pd.factorize(dataframe_log['id'])
dataframe_log['id'] = codes + 1

In [5]:
codes, uniques = pd.factorize(dataframe_log['org:resource'])
dataframe_log['org:resource'] = codes + 1

In [6]:
codes, uniques = pd.factorize(dataframe_log['concept:name'])
dataframe_log['concept:name'] = codes + 1

In [7]:
codes, uniques = pd.factorize(dataframe_log['org:role'])
dataframe_log['org:role'] = codes + 1

# Generate Prefixes

In [8]:
df_id = dataframe_log[['id', '@@case_index']]
df_resource = dataframe_log[['org:resource', '@@case_index']]
df_activity = dataframe_log[['concept:name', '@@case_index']]
df_role = dataframe_log[['org:role', '@@case_index']]
df_timestamp = dataframe_log[['standardized_elapsed_time', '@@case_index']]

In [9]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_prefix_windows(df, case_id_column='@@case_index', max_len=None):
    windows = []
    targets = []
    case_indices = []
    
    for case_id in df[case_id_column].unique():
        case_data = df[df[case_id_column] == case_id].drop(columns=[case_id_column]).to_numpy()
        
        # Optional: Make sure to sort the case data if there's an implicit order (e.g., by timestamps)
        # case_data = case_data.sort_values(by='timestamp_column').to_numpy()  # Uncomment and adjust if needed
        
        for i in range(1, len(case_data)):
            window = case_data[:i]
            target = case_data[i]
            windows.append(window)
            targets.append(target)
            case_indices.append(case_id)
    
    if max_len is None:
        max_len = max(len(window) for window in windows)
    
    # Pad sequences
    windows_padded = pad_sequences(windows, maxlen=max_len, padding='post', dtype='float32')
    
    # Convert targets to numpy array
    targets_array = np.array(targets, dtype='float32')
    case_indices_array = np.array(case_indices)
    
    return windows_padded, targets_array, case_indices_array

2024-08-18 18:02:18.516354: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
windows_id, targets_id, case_indices = generate_prefix_windows(df_id)
windows_resource, targets_resource, case_indices = generate_prefix_windows(df_resource)
windows_activity, targets_activity, case_indices = generate_prefix_windows(df_activity)
windows_role, targets_role, case_indices = generate_prefix_windows(df_role)
windows_timestamp, targets_timestamp, case_indices = generate_prefix_windows(df_timestamp)

# GRU

### Architecture

- Separate Inputs for Each Attribute
- Each attribute is passed through an embedding layer
- Each attribute has its corresponding GRU encoder
- Selective Concatenation: After encoding, the outputs of these GRU layers are concatenated. However, this concatenation is selective, meaning it is structured in a way that prepares the data for effective synthesis without leaking information from the future (next event attributes)
- Decoder GRUs: Integrated Decoding: Post-concatenation, the combined attributes are processed through decoder GRU layers. These layers are tasked with integrating the data from different attributes and preparing it for final prediction. This step is where BINet v3 distinguishes itself by effectively using the interdependencies between different attributes to enhance prediction accuracy.
- Output Layer: Softmax Output for Each Attribute: For each attribute of the next event, a softmax layer predicts a probability distribution over all possible values. This allows the model to output the most likely next event and its attributes based on the learned dependencies and the history encoded by the GRUs.
- E: maximum case length
- We train BINet with a GRU size of 2E (two times the maximum case length)
- on mini batches of size 500 for 20 epochs

In [11]:
# Group by the @@case_index column and count the rows in each group
case_lengths = dataframe_log.groupby('@@case_index').size()

# Find the maximum value among the case lengths
E = case_lengths.max()

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Embedding, Dense, Dropout, Concatenate, BatchNormalization

def create_binetv3(num_ids, num_resources, num_activities, num_roles, embedding_dim, gru_units, dropout_rate):
    # Input layers for each attribute
    input_id = Input(shape=(None,), name='id_input')
    input_resource = Input(shape=(None,), name='resource_input')
    input_activity = Input(shape=(None,), name='activity_input')
    input_role = Input(shape=(None,), name='role_input')
    input_timestamp = Input(shape=(None, 1), name='timestamp_input')

    # Embedding layers for categorical attributes
    embedding_id = Embedding(input_dim=num_ids + 1, output_dim=embedding_dim, mask_zero=True, name='id_embedding')(input_id)
    embedding_resource = Embedding(input_dim=num_resources + 1, output_dim=embedding_dim, mask_zero=True, name='resource_embedding')(input_resource)
    embedding_activity = Embedding(input_dim=num_activities + 1, output_dim=embedding_dim, mask_zero=True, name='activity_embedding')(input_activity)
    embedding_role = Embedding(input_dim=num_roles + 1, output_dim=embedding_dim, mask_zero=True, name='role_embedding')(input_role)

    # Encoder GRUs with Batch Normalization for categorical attributes
    encoded_id = GRU(units=gru_units, return_sequences=True, name='id_encoder')(embedding_id)
    bn_id = BatchNormalization(name='bn_id')(encoded_id)
    encoded_resource = GRU(units=gru_units, return_sequences=True, name='resource_encoder')(embedding_resource)
    bn_resource = BatchNormalization(name='bn_resource')(encoded_resource)
    encoded_activity = GRU(units=gru_units, return_sequences=True, name='activity_encoder')(embedding_activity)
    bn_activity = BatchNormalization(name='bn_activity')(encoded_activity)
    encoded_role = GRU(units=gru_units, return_sequences=True, name='role_encoder')(embedding_role)
    bn_role = BatchNormalization(name='bn_role')(encoded_role)

    # Encoder GRU with Batch Normalization for continuous attribute
    encoded_timestamp = GRU(units=gru_units, return_sequences=True, name='timestamp_encoder')(input_timestamp)
    bn_timestamp = BatchNormalization(name='bn_timestamp')(encoded_timestamp)

    # Concatenation of encoded outputs
    concatenated = Concatenate(name='concatenate_encodings')([bn_id, bn_resource, bn_activity, bn_role, bn_timestamp])

    # Decoder GRU
    decoder_output = GRU(units=gru_units, return_sequences=False, name='decoder_gru')(concatenated)
    dropout_layer = Dropout(rate=dropout_rate, name='dropout')(decoder_output)

    # Output layers for predicting the next event's attributes
    output_id = Dense(num_ids + 1, activation='softmax', name='output_id')(dropout_layer)
    output_resource = Dense(num_resources + 1, activation='softmax', name='output_resource')(dropout_layer)
    output_activity = Dense(num_activities + 1, activation='softmax', name='output_activity')(dropout_layer)
    output_role = Dense(num_roles + 1, activation='softmax', name='output_role')(dropout_layer)
    output_timestamp = Dense(1, activation='linear', name='output_timestamp')(dropout_layer)

    # Building the model
    model = Model(inputs=[input_id, input_resource, input_activity, input_role, input_timestamp], outputs=[output_id, output_resource, output_activity, output_role, output_timestamp])
    model.compile(
        optimizer='adam', 
        loss={
            'output_id': 'categorical_crossentropy',
            'output_resource': 'categorical_crossentropy', 
            'output_activity': 'categorical_crossentropy',
            'output_role': 'categorical_crossentropy',
            'output_timestamp': 'mse'
        },
        metrics={
            'output_id': ['accuracy'],
            'output_resource': ['accuracy'], 
            'output_activity': ['accuracy'],
            'output_role': ['accuracy'],
            'output_timestamp': ['mse']
        }
    )

    return model

# Parameters
gru_units = int(2 * E) 
num_ids = dataframe_log['id'].max()
num_resources = dataframe_log['org:resource'].max()
num_activities = dataframe_log['concept:name'].max()
num_roles = dataframe_log['org:role'].max()
embedding_dim = 50
dropout_rate = 0.2
model = create_binetv3(num_ids, num_resources, num_activities, num_roles, embedding_dim, gru_units, dropout_rate)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 id_input (InputLayer)       [(None, None)]               0         []                            
                                                                                                  
 resource_input (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 activity_input (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                              

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Embedding, Dense, Dropout, Concatenate, BatchNormalization

def create_binetv3(num_resources, num_activities, num_roles, embedding_dim, gru_units, dropout_rate):
    # Input layers for each attribute
    input_resource = Input(shape=(None,), name='resource_input')
    input_activity = Input(shape=(None,), name='activity_input')
    input_role = Input(shape=(None,), name='role_input')
    input_timestamp = Input(shape=(None, 1), name='timestamp_input')

    # Embedding layers for categorical attributes
    embedding_resource = Embedding(input_dim=num_resources + 1, output_dim=embedding_dim, mask_zero=True, name='resource_embedding')(input_resource)
    embedding_activity = Embedding(input_dim=num_activities + 1, output_dim=embedding_dim, mask_zero=True, name='activity_embedding')(input_activity)
    embedding_role = Embedding(input_dim=num_roles + 1, output_dim=embedding_dim, mask_zero=True, name='role_embedding')(input_role)

    # Encoder GRUs with Batch Normalization for categorical attributes
    encoded_resource = GRU(units=gru_units, return_sequences=True, name='resource_encoder')(embedding_resource)
    bn_resource = BatchNormalization(name='bn_resource')(encoded_resource)
    encoded_activity = GRU(units=gru_units, return_sequences=True, name='activity_encoder')(embedding_activity)
    bn_activity = BatchNormalization(name='bn_activity')(encoded_activity)
    encoded_role = GRU(units=gru_units, return_sequences=True, name='role_encoder')(embedding_role)
    bn_role = BatchNormalization(name='bn_role')(encoded_role)

    # Encoder GRU with Batch Normalization for continuous attribute
    encoded_timestamp = GRU(units=gru_units, return_sequences=True, name='timestamp_encoder')(input_timestamp)
    bn_timestamp = BatchNormalization(name='bn_timestamp')(encoded_timestamp)

    # Concatenation of encoded outputs
    concatenated = Concatenate(name='concatenate_encodings')([bn_resource, bn_activity, bn_role, bn_timestamp])

    # Decoder GRU
    decoder_output = GRU(units=gru_units, return_sequences=False, name='decoder_gru')(concatenated)
    dropout_layer = Dropout(rate=dropout_rate, name='dropout')(decoder_output)

    # Output layers for predicting the next event's attributes
    output_resource = Dense(num_resources + 1, activation='softmax', name='output_resource')(dropout_layer)
    output_activity = Dense(num_activities + 1, activation='softmax', name='output_activity')(dropout_layer)
    output_role = Dense(num_roles + 1, activation='softmax', name='output_role')(dropout_layer)
    output_timestamp = Dense(1, activation='linear', name='output_timestamp')(dropout_layer)

    # Building the model
    model = Model(inputs=[input_resource, input_activity, input_role, input_timestamp], outputs=[output_resource, output_activity, output_role, output_timestamp])
    model.compile(
        optimizer='adam', 
        loss={
            'output_resource': 'categorical_crossentropy', 
            'output_activity': 'categorical_crossentropy',
            'output_role': 'categorical_crossentropy',
            'output_timestamp': 'mse'
        },
        metrics={
            'output_resource': ['accuracy'], 
            'output_activity': ['accuracy'],
            'output_role': ['accuracy'],
            'output_timestamp': ['mse']
        }
    )

    return model

# Parameters
gru_units = int(2 * E) 
num_resources = dataframe_log['org:resource'].max()
num_activities = dataframe_log['concept:name'].max()
num_roles = dataframe_log['org:role'].max()
embedding_dim = 50
dropout_rate = 0.2
model = create_binetv3(num_resources, num_activities, num_roles, embedding_dim, gru_units, dropout_rate)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 resource_input (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 activity_input (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 role_input (InputLayer)     [(None, None)]               0         []                            
                                                                                            

In [14]:
num_resources

2

In [15]:
num_roles

8

### Data Splitting

In [16]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_id, test_id, train_targets_id, test_targets_id = train_test_split(
    windows_id, targets_id, test_size=0.3, random_state=42)

train_resource, test_resource, train_targets_resource, test_targets_resource = train_test_split(
    windows_resource, targets_resource, test_size=0.3, random_state=42)

train_activity, test_activity, train_targets_activity, test_targets_activity = train_test_split(
    windows_activity, targets_activity, test_size=0.3, random_state=42)

train_role, test_role, train_targets_role, test_targets_role = train_test_split(
    windows_role, targets_role, test_size=0.3, random_state=42)

train_timestamp, test_timestamp, train_targets_timestamp, test_targets_timestamp = train_test_split(
    windows_timestamp, targets_timestamp, test_size=0.3, random_state=42)

### Training

In [17]:
from tensorflow.keras.utils import to_categorical

train_targets_resource_cat = to_categorical(train_targets_resource, num_classes=num_resources + 1)
test_targets_resource_cat = to_categorical(test_targets_resource, num_classes=num_resources + 1)

train_targets_activity_cat = to_categorical(train_targets_activity, num_classes=num_activities + 1)
test_targets_activity_cat = to_categorical(test_targets_activity, num_classes=num_activities + 1)

train_targets_role_cat = to_categorical(train_targets_role, num_classes=num_roles + 1)
test_targets_role_cat = to_categorical(test_targets_role, num_classes=num_roles + 1)

In [18]:
# Train the model with the properly shaped targets
history = model.fit(
    [train_resource, train_activity, train_role, train_timestamp],
    [train_targets_resource_cat, train_targets_activity_cat, train_targets_role_cat, train_targets_timestamp],
    validation_data=(
        [test_resource, test_activity, test_role, test_timestamp], 
        [test_targets_resource_cat, test_targets_activity_cat, test_targets_role_cat, test_targets_timestamp]
    ),
    epochs=20,
    batch_size=500
)

Epoch 1/20
92/92 [==============================] - 53s 364ms/step - loss: 3.2319 - output_resource_loss: 0.2299 - output_activity_loss: 1.7170 - output_role_loss: 0.9228 - output_timestamp_loss: 0.3622 - output_resource_accuracy: 0.9347 - output_activity_accuracy: 0.6183 - output_role_accuracy: 0.7362 - output_timestamp_mse: 0.3622 - val_loss: 5.8399 - val_output_resource_loss: 0.5447 - val_output_activity_loss: 2.8565 - val_output_role_loss: 1.5978 - val_output_timestamp_loss: 0.8409 - val_output_resource_accuracy: 0.8967 - val_output_activity_accuracy: 0.6198 - val_output_role_accuracy: 0.7733 - val_output_timestamp_mse: 0.8409
Epoch 2/20
92/92 [==============================] - 30s 322ms/step - loss: 1.4433 - output_resource_loss: 0.0662 - output_activity_loss: 0.7320 - output_role_loss: 0.3758 - output_timestamp_loss: 0.2694 - output_resource_accuracy: 0.9901 - output_activity_accuracy: 0.8152 - output_role_accuracy: 0.8865 - output_timestamp_mse: 0.2694 - val_loss: 4.2376 - val_o

In [19]:
# Evaluate the model on the validation set
results = model.evaluate(
    [test_resource, test_activity, test_role, test_timestamp],
    [test_targets_resource_cat, test_targets_activity_cat, test_targets_role_cat, test_targets_timestamp],
    batch_size=64
)
print(f"Validation Loss: {results[0]}, Validation Accuracy: {results[1]}")

308/308 [==============================] - 8s 25ms/step - loss: 0.7546 - output_resource_loss: 0.0361 - output_activity_loss: 0.3968 - output_role_loss: 0.2261 - output_timestamp_loss: 0.0956 - output_resource_accuracy: 0.9905 - output_activity_accuracy: 0.8695 - output_role_accuracy: 0.9159 - output_timestamp_mse: 0.0956
Validation Loss: 0.7546376585960388, Validation Accuracy: 0.03609570488333702


In [20]:
# Save the model to an H5 file
model.save('International.h5')

/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Anomaly Score Computation

- For each event attribute, BINet's softmax layer outputs a probability distribution over possible values
- The anomaly score for a specific attribute value v is calculated by summing all the probabilities from the softmax output that are greater than the probability assigned to v

In [21]:
# Generate predictions for all inputs
predictions = model.predict([windows_resource, windows_activity, windows_role, windows_timestamp])


# Assuming your model is set to predict resource, activity, and role categories
# Extract predictions for categorical attributes (softmax probabilities)
predictions_activity = predictions[1]  # Resource predictions
predictions_resource = predictions[0]  # Activity predictions
predictions_role = predictions[2]      # Role predictions

# If you had added 'amount' as a target to be predicted, you would extract its predictions like so:
predictions_timestamp = predictions[3]  # Assuming 'amount' is a regression target and the model is adjusted accordingly

2054/2054 [==============================] - 48s 19ms/step


In [22]:
import numpy as np

def calculate_anomaly_scores(predictions, targets):
    scores = []
    # Loop through each example in the predictions
    for i in range(predictions.shape[0]):
        actual_prob = predictions[i, targets[i]]  # Extract the probability of the true class using target index
        # Calculate anomaly score as sum of probabilities greater than the probability of the actual value
        anomaly_score = np.sum(predictions[i][predictions[i] > actual_prob])
        scores.append(anomaly_score)

    return scores

In [23]:
anomaly_scores_resource = calculate_anomaly_scores(predictions_resource, targets_resource.astype(int))
anomaly_scores_role = calculate_anomaly_scores(predictions_role, targets_role.astype(int))
anomaly_scores_activity = calculate_anomaly_scores(predictions_activity, targets_activity.astype(int))

In [24]:
def compute_anomaly_scores_continuous(predictions, actuals):
    # Calculate absolute differences
    differences = np.abs(predictions - actuals)
    
    # Normalize to [0, 1] range
    max_diff = np.max(differences)
    normalized_scores = differences / max_diff if max_diff != 0 else differences
    
    return normalized_scores

In [25]:
anomaly_scores_timestamp = compute_anomaly_scores_continuous(predictions_timestamp, targets_timestamp)
anomaly_scores_timestamp = anomaly_scores_timestamp.flatten()
anomaly_scores_timestamp = anomaly_scores_timestamp.tolist()

## Insert missing scores for cases with less than 2 Events

In [26]:
import pandas as pd

# Create a DataFrame from the case_indices_array corresponding to case_resource
score = pd.DataFrame({'case': case_indices})
score['score_resource'] = anomaly_scores_resource
score['score_activity'] = anomaly_scores_activity
score['score_role'] = anomaly_scores_role
score['score_timestamp'] = anomaly_scores_timestamp

score

,case,score_resource,score_activity,score_role,score_timestamp
0,0,0.0,0.000000,0.000000,0.031133
1,0,0.0,0.000000,0.000000,0.364989
2,0,0.0,0.945747,0.981951,0.027820
3,0,0.0,0.000000,0.000000,0.064265
4,0,0.0,0.000000,0.000000,0.006259
...,...,...,...,...,...
65697,6448,0.0,0.963246,0.000000,0.076238
65698,6448,0.0,0.745861,0.000000,0.028273
65699,6448,0.0,0.000000,0.000000,0.015058
65700,6448,0.0,0.000000,0.000000,0.062341


In [27]:
import pandas as pd

def contains_all_values(df, column, end):

    # Generate the set of all values in the specified range
    required_values = set(range(0, end + 1))
    
    # Get the unique values in the specified column
    column_values = set(df[column].unique())
    
    # Find missing values
    missing_values = required_values - column_values
    
    # Print missing values if any
    if missing_values:
        print(f"Missing values: {sorted(missing_values)}")
    
    # Check if all required values are in the column values
    return required_values.issubset(column_values)

end = 6448

result = contains_all_values(score, 'case', end)
print(f"Does the 'case' column contain all values between 0 and {end}? {result}")

Does the 'case' column contain all values between 0 and 6448? True


### Threshold (lowest plateau)

In [28]:
import numpy as np

def calculate_anomaly_ratio(scores, threshold):
    """
    Calculate the anomaly ratio for a given threshold.
    """
    return np.mean(scores > threshold)

def find_plateaus(scores, epsilon=1e-4, min_plateau_length=10):
    """
    Identify the lowest plateau in the anomaly ratio function and calculate the mean-centered threshold.
    """
    scores = np.array(scores)  # Convert scores to a NumPy array
    sorted_scores = np.sort(scores)
    
    # Remove duplicate values
    unique_thresholds, unique_indices = np.unique(sorted_scores, return_index=True)
    anomaly_ratios = np.array([calculate_anomaly_ratio(scores, t) for t in unique_thresholds])
    
    # Calculate first and second derivatives
    first_derivatives = np.diff(anomaly_ratios) / np.diff(unique_thresholds)
    second_derivatives = np.diff(first_derivatives) / np.diff(unique_thresholds[:-1])
    
    # Identify plateaus where the first derivative is close to zero
    plateau_indices = np.where(np.abs(first_derivatives) < epsilon)[0]
    
    # Group consecutive indices to identify continuous plateaus
    grouped_plateaus = np.split(plateau_indices, np.where(np.diff(plateau_indices) != 1)[0] + 1)
    
    # Filter plateaus based on minimum length
    long_plateaus = [g for g in grouped_plateaus if len(g) >= min_plateau_length]
    
    if long_plateaus:
        # Take the first long plateau and find the mean threshold in this plateau
        first_plateau = long_plateaus[0]
        plateau_thresholds = unique_thresholds[first_plateau]
        return np.mean(plateau_thresholds)
    else:
        # If no plateau is found, return a default value, e.g., the 90th percentile
        percentile_90 = np.percentile(sorted_scores, 90)
        if percentile_90 == 1.0:
            return 0.4
        else:
            return percentile_90

In [29]:
threshold_resource = find_plateaus(anomaly_scores_resource)
threshold_role = find_plateaus(anomaly_scores_role)
threshold_activity = find_plateaus(anomaly_scores_activity)
threshold_timestamp = find_plateaus(anomaly_scores_timestamp)

### Detection

In [30]:
def detect_anomalies(anomaly_scores, threshold):
    labels = [1 if score > threshold else 0 for score in anomaly_scores]
    return labels

In [31]:
# Detect anomalies based on the calculated anomaly scores and thresholds
labels_resource = detect_anomalies(anomaly_scores_resource, threshold_resource)
labels_activity = detect_anomalies(anomaly_scores_activity, threshold_activity)
labels_role = detect_anomalies(anomaly_scores_role, threshold_role)
labels_timestamp = detect_anomalies(anomaly_scores_timestamp, threshold_timestamp)

# Mapping

In [32]:
import pandas as pd

# Create a DataFrame from the case_indices_array corresponding to case_resource
mapping = pd.DataFrame({'case': case_indices})
mapping['predicted_resource'] = labels_resource
mapping['predicted_activity'] = labels_activity
mapping['predicted_role'] = labels_role
mapping['predicted_timestamp'] = labels_timestamp

mapping

,case,predicted_resource,predicted_activity,predicted_role,predicted_timestamp
0,0,0,0,0,0
1,0,0,0,0,1
2,0,0,1,1,0
3,0,0,0,0,1
4,0,0,0,0,0
...,...,...,...,...,...
65697,6448,0,1,0,1
65698,6448,0,1,0,0
65699,6448,0,0,0,0
65700,6448,0,0,0,1


In [33]:
# Create a boolean DataFrame where each value is True if the value is 1
contains_one = (mapping[['predicted_resource', 'predicted_activity', 'predicted_role', 'predicted_timestamp']] == 1)

# Group by 'case' and check if there's at least one 'True' in any of the columns
case_prediction = contains_one.groupby(mapping['case']).any().any(axis=1)
case_prediction

case
0       True
1       True
2       True
3       True
4       True
        ... 
6444    True
6445    True
6446    True
6447    True
6448    True
Length: 6449, dtype: bool

# Ground Truth

In [34]:
def generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking):
    from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments
    from pm4py.algo.conformance.alignments.petri_net import variants
    from pm4py.objects.petri_net.utils import align_utils
    max_events=0
    for trace in log:
        counter=0
        for event in trace:
            counter+=1
        if counter > max_events:
            max_events=counter
    parameters={}
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_SYNC_COST_FUNCTION] = list(map(lambda i: .1*i, range(max_events*2)))
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_TRACE_COST_FUNCTION]=list(map(lambda i: align_utils.STD_MODEL_LOG_MOVE_COST-.1*i, range(max_events*2)))
    aligned_traces = alignments.apply_log(log, net, initial_marking, final_marking, variant=variants.state_equation_a_star, parameters=parameters)
    return aligned_traces

In [35]:
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.bpmn.importer import importer as bpmn_importer
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments_petri

# 2. Import the given BPMN model
bpmn_graph = bpmn_importer.apply("../../data/model/Model_InternationalDeclarations.bpmn")

# 3. Convert the BPMN to a Petri net
net, initial_marking, final_marking = pm4py.convert_to_petri_net(bpmn_graph)

aligned_traces = generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking)

aligning log, completed variants ::   0%|          | 0/753 [00:00<?, ?it/s]

In [36]:
def extract_conformance_status_by_fitness(aligned_traces):
    conformance_status = []
    for alignment in aligned_traces:
        fitness = alignment['fitness']
        # If the fitness is 1.0, the trace is conforming
        if fitness == 1.0:
            conformance_status.append(0)
        else:
            conformance_status.append(1)
    return conformance_status

# Get the conformance status list from the aligned traces
conformance = extract_conformance_status_by_fitness(aligned_traces)

In [37]:
ground_truth = pd.DataFrame({'conformity': conformance})
ground_truth['predicted'] = case_prediction

# Convert False to 0 and True to 1
ground_truth['predicted'] = [int(value) for value in ground_truth['predicted']]
ground_truth

,conformity,predicted
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
...,...,...
6444,0,1
6445,1,1
6446,0,1
6447,1,1


# Evaluation

In [38]:
# Calculating TP, TN, FP, FN
TP = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 1)).sum()
TN = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 0)).sum()
FP = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 1)).sum()
FN = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 0)).sum()

In [39]:
precision_dev = TP / (TP + FP)
print(f"Precision Dev: {precision_dev:.2f}")

Precision Dev: 0.65


In [40]:
recall_dev = TP / (TP + FN)
print(f"Recall Dev: {recall_dev:.2f}")

Recall Dev: 0.98


In [41]:
precision_no_dev = TN / (TN + FN)
print(f"Precision No Dev: {precision_no_dev:.2f}")

Precision No Dev: 0.87


In [42]:
recall_no_dev = TN / (TN + FP)
print(f"Recall No Dev: {recall_no_dev:.2f}")

Recall No Dev: 0.17


In [43]:
from sklearn.metrics import roc_auc_score

auc_roc = roc_auc_score(ground_truth['conformity'], ground_truth['predicted'])
print(f"AUC-ROC: {auc_roc:.2f}")

AUC-ROC: 0.58
